# Neuron Network - Lab

### Part 1: Load  data

Import "bank-data.csv"

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
bankData = pd.read_csv('/content/gdrive/MyDrive/CPE/CPE378 Machine Learning/bank-data.csv', sep = ';')
bankData.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


### Part 2: Preprocess data

Preprocess the dataset as you have done before

#### 2.1 Binary encoding

Use LabelEncoder to encode the following columns:
- y
- default
- housing
- loan

In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

#example
bankData['y'] = le.fit_transform(bankData['y'])
bankData.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,0
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,0


In [4]:
#Encode the remaining columns
bankData['housing'] = le.fit_transform(bankData['housing'])
bankData['default'] = le.fit_transform(bankData['default'])
bankData['loan'] = le.fit_transform(bankData['loan'])
bankData.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,0,1787,0,0,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,0,4789,1,1,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,0,1350,1,0,cellular,16,apr,185,1,330,1,failure,0
3,30,management,married,tertiary,0,1476,1,1,unknown,3,jun,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,0,0,1,0,unknown,5,may,226,1,-1,0,unknown,0


#### 2.2 Convert categorical variables into dummy columns

(1) Use pd.get_dummies to convert the following categorical variales into dummy columns
- job
- maritial
- education
- contact
- month
- poutcome

(2) Drop columns that have been converted

In [5]:
#example
bankData = pd.concat([bankData,pd.get_dummies(bankData['job'],prefix='job')],axis=1)
bankData = bankData.drop(columns=['job'])
bankData.head()

,age,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown
0,30,married,primary,0,1787,0,0,cellular,19,oct,79,1,-1,0,unknown,0,0,0,0,0,0,0,0,0,0,0,1,0
1,33,married,secondary,0,4789,1,1,cellular,11,may,220,1,339,4,failure,0,0,0,0,0,0,0,0,1,0,0,0,0
2,35,single,tertiary,0,1350,1,0,cellular,16,apr,185,1,330,1,failure,0,0,0,0,0,1,0,0,0,0,0,0,0
3,30,married,tertiary,0,1476,1,1,unknown,3,jun,199,4,-1,0,unknown,0,0,0,0,0,1,0,0,0,0,0,0,0
4,59,married,secondary,0,0,1,0,unknown,5,may,226,1,-1,0,unknown,0,0,1,0,0,0,0,0,0,0,0,0,0


In [6]:
bankData = pd.concat([bankData,pd.get_dummies(bankData['marital'],prefix='marital')],axis=1)
bankData = pd.concat([bankData,pd.get_dummies(bankData['education'],prefix='education')],axis=1)
bankData = pd.concat([bankData,pd.get_dummies(bankData['contact'],prefix='contact')],axis=1)
bankData = pd.concat([bankData,pd.get_dummies(bankData['month'],prefix='month')],axis=1)
bankData = pd.concat([bankData,pd.get_dummies(bankData['poutcome'],prefix='poutcome')],axis=1)

bankData = bankData.drop(columns=['marital', 'education', 'contact', 'month', 'poutcome'])

In [7]:
# all column is integer
bankData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 49 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   age                  4521 non-null   int64
 1   default              4521 non-null   int64
 2   balance              4521 non-null   int64
 3   housing              4521 non-null   int64
 4   loan                 4521 non-null   int64
 5   day                  4521 non-null   int64
 6   duration             4521 non-null   int64
 7   campaign             4521 non-null   int64
 8   pdays                4521 non-null   int64
 9   previous             4521 non-null   int64
 10  y                    4521 non-null   int64
 11  job_admin.           4521 non-null   uint8
 12  job_blue-collar      4521 non-null   uint8
 13  job_entrepreneur     4521 non-null   uint8
 14  job_housemaid        4521 non-null   uint8
 15  job_management       4521 non-null   uint8
 16  job_retired          452

#### 2.3 Train/Test separation

Perform hold-out method
- 60% training set
- 40% testing set

In [8]:
bankData_train = bankData.sample(frac = 0.6)
bankData_test = bankData.drop(bankData_train.index)
print(pd.crosstab(bankData_train['y'],columns = 'count'))
print(pd.crosstab(bankData_test['y'],columns = 'count'))

col_0  count
y           
0       2394
1        319
col_0  count
y           
0       1606
1        202


##### X/y separation

In [10]:
bankData_train_y = bankData_train['y']
bankData_train_X = bankData_train.copy()
del bankData_train_X['y']

bankData_test_y = bankData_test['y']
bankData_test_X = bankData_test.copy()
del bankData_test_X['y']

#### 2.4 Feature Scaling

It is always a good practice to scale the features so that all of them can be uniformly evaluated

In [11]:
from sklearn import preprocessing

#standardization
standard_scaler = preprocessing.StandardScaler()
train_X_scaled_s = pd.DataFrame(standard_scaler.fit_transform(bankData_train_X), columns=bankData_train_X.columns)
test_X_scaled_s = pd.DataFrame(standard_scaler.fit_transform(bankData_test_X), columns=bankData_train_X.columns)

#normalization
min_max_scaler = preprocessing.MinMaxScaler()
train_X_scaled_m = pd.DataFrame(min_max_scaler.fit_transform(bankData_train_X),columns=bankData_train_X.columns)
test_X_scaled_m = pd.DataFrame(min_max_scaler.fit_transform(bankData_test_X),columns=bankData_train_X.columns)

In [12]:
train_X_scaled_s.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,0.701541,-0.134206,0.636037,-1.127916,-0.432792,-0.576458,0.024052,-0.259288,-0.401466,-0.305667,-0.347957,-0.496773,-0.198654,-0.16034,1.919809,-0.24447,-0.209435,-0.329016,-0.141155,-0.441763,-0.177648,-0.090418,-0.363086,0.772542,-0.588559,2.387209,-1.012985,-0.653562,-0.216993,0.734662,-0.266575,-0.630090,-0.250342,-0.412802,-0.074563,-0.2315,-0.180934,-0.424356,-0.36633,-0.110966,-0.662765,3.210010,-0.139791,-0.107511,-0.337211,-0.219772,-0.169766,0.462461
1,0.241523,-0.134206,-0.694859,0.886591,-0.432792,-0.697515,-0.304164,-0.588003,-0.401466,-0.305667,-0.347957,-0.496773,-0.198654,-0.16034,1.919809,-0.24447,-0.209435,-0.329016,-0.141155,-0.441763,-0.177648,-0.090418,2.754168,-1.294427,-0.588559,-0.418899,-1.012985,1.530076,-0.216993,0.734662,-0.266575,-0.630090,-0.250342,-0.412802,-0.074563,-0.2315,-0.180934,2.356510,-0.36633,-0.110966,-0.662765,-0.311525,-0.139791,-0.107511,-0.337211,-0.219772,-0.169766,0.462461
2,-0.678515,-0.134206,-0.002832,0.886591,2.310581,1.360448,-0.403393,-0.259288,-0.401466,-0.305667,-0.347957,2.012993,-0.198654,-0.16034,-0.520885,-0.24447,-0.209435,-0.329016,-0.141155,-0.441763,-0.177648,-0.090418,-0.363086,0.772542,-0.588559,-0.418899,0.987181,-0.653562,-0.216993,-1.361171,-0.266575,1.587075,-0.250342,-0.412802,-0.074563,-0.2315,-0.180934,-0.424356,-0.36633,-0.110966,1.508831,-0.311525,-0.139791,-0.107511,-0.337211,-0.219772,-0.169766,0.462461
3,-1.506549,-0.134206,-0.452226,-1.127916,2.310581,-1.423855,0.203427,-0.588003,-0.401466,-0.305667,-0.347957,-0.496773,-0.198654,-0.16034,-0.520885,-0.24447,-0.209435,-0.329016,-0.141155,-0.441763,5.629098,-0.090418,-0.363086,-1.294427,1.699064,-0.418899,0.987181,-0.653562,-0.216993,-1.361171,-0.266575,1.587075,-0.250342,-0.412802,-0.074563,-0.2315,-0.180934,2.356510,-0.36633,-0.110966,-0.662765,-0.311525,-0.139791,-0.107511,-0.337211,-0.219772,-0.169766,0.462461
4,-1.046530,-0.134206,-0.463314,0.886591,-0.432792,0.997278,-0.204936,-0.259288,-0.401466,-0.305667,2.873915,-0.496773,-0.198654,-0.16034,-0.520885,-0.24447,-0.209435,-0.329016,-0.141155,-0.441763,-0.177648,-0.090418,-0.363086,0.772542,-0.588559,-0.418899,-1.012985,1.530076,-0.216993,0.734662,-0.266575,-0.630090,-0.250342,-0.412802,-0.074563,-0.2315,-0.180934,2.356510,-0.36633,-0.110966,-0.662765,-0.311525,-0.139791,-0.107511,-0.337211,-0.219772,-0.169766,0.462461


In [13]:
# normalize to range 0-1
train_X_scaled_m.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,0.441176,0.0,0.074423,0.0,0.0,0.333333,0.087417,0.032258,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.367647,0.0,0.018725,1.0,0.0,0.300000,0.058940,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.220588,0.0,0.047687,1.0,1.0,0.866667,0.050331,0.032258,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.088235,0.0,0.028879,0.0,1.0,0.100000,0.102980,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.161765,0.0,0.028415,1.0,0.0,0.766667,0.067550,0.032258,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Artificial Neural Network : sklearn

### Part 3: Train a model

In [14]:
from sklearn.neural_network import MLPClassifier
# 10 neurons per layers  
mlp = MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000)  # max_iter = epoch
mlp.fit(train_X_scaled_s, bankData_train_y) 

MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000)

### Part 4: Model Evaluation

Evaluation metrics
- confusion metrix
- accuracy
- precision, recall, f1-score

In [28]:
mlp.score(train_X_scaled_s, bankData_train_y, sample_weight=None)

0.9944710652414301

### Part 5: Model tuning

#### Note:

After building the classifier, try answering the following questions.

1. What is the Accuracy Score?
2. If you change your preprosessing method, can you improve the model?
3. If you change your parameters setting, can you improve the model?

## Artificial Neural Network : keras

Fitting a logistic regression model

### Part 3: Train a model

In [15]:
import keras
from keras import models
from keras import layers

In [16]:
train_X_scaled_s.shape

(2713, 48)

In [18]:
nn = models.Sequential()
# 48 neurons in input layer
nn.add(layers.Dense(48,activation = 'linear',input_shape=(None,48)))
# output is 0/1
nn.add(layers.Dense(1,activation = 'sigmoid'))
# Dense = fully connected network

In [19]:
nn.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, None, 48)          2352      
                                                                 
 dense_3 (Dense)             (None, None, 1)           49        
                                                                 
Total params: 2,401
Trainable params: 2,401
Non-trainable params: 0
_________________________________________________________________


In [23]:
import numpy as np
X_train_add = np.expand_dims(train_X_scaled_s, axis=0)
y_train_add = np.expand_dims(bankData_train_y, axis=0)
y_train_add = np.expand_dims(y_train_add, axis=2)

In [ ]:
y_train_add.shape

In [25]:
history = nn.fit(X_train_add,y_train_add,epochs=100)

Epoch 1/100
1/1 [==============================] - 1s 766ms/step - loss: 0.7654 - accuracy: 0.5459
Epoch 2/100
1/1 [==============================] - 0s 12ms/step - loss: 0.7599 - accuracy: 0.5511
Epoch 3/100
1/1 [==============================] - 0s 11ms/step - loss: 0.7544 - accuracy: 0.5555
Epoch 4/100
1/1 [==============================] - 0s 15ms/step - loss: 0.7490 - accuracy: 0.5584
Epoch 5/100
1/1 [==============================] - 0s 14ms/step - loss: 0.7437 - accuracy: 0.5628
Epoch 6/100
1/1 [==============================] - 0s 14ms/step - loss: 0.7385 - accuracy: 0.5673
Epoch 7/100
1/1 [==============================] - 0s 10ms/step - loss: 0.7333 - accuracy: 0.5743
Epoch 8/100
1/1 [==============================] - 0s 11ms/step - loss: 0.7282 - accuracy: 0.5798
Epoch 9/100
1/1 [==============================] - 0s 9ms/step - loss: 0.7232 - accuracy: 0.5827
Epoch 10/100
1/1 [==============================] - 0s 11ms/step - loss: 0.7183 - accuracy: 0.5864
Epoch 11/100
1/1 [=

### Part 4: Model Evaluation

In [26]:
X_test_add = np.expand_dims(test_X_scaled_s, axis=0)
y_test_add = np.expand_dims(bankData_test_y, axis=0)
y_test_add = np.expand_dims(y_test_add, axis=2)

In [27]:
test_loss, test_acc = nn.evaluate(X_test_add, y_test_add)
print('Test Loss: %s\nTest Accuracy: %s' % (test_loss,test_acc))

1/1 [==============================] - 0s 175ms/step - loss: 0.4510 - accuracy: 0.8711
Test Loss: 0.4510183334350586
Test Accuracy: 0.8711283206939697


### Part 5: Model tuning

#### Note:

After building the classifier, try answering the following questions.

1. What is the Accuracy Score?
2. If you change your preprosessing method, can you improve the model?
3. If you change your parameters setting, can you improve the model?